In [1]:
from tqdm import tqdm
import requests
import pandas as pd
import re
from unicodedata import normalize
from time import sleep
from datetime import datetime
import numpy as np
from geopy.geocoders import Nominatim

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

import warnings
warnings.filterwarnings('ignore')
import src.support as sp 

In [4]:
# Takes the path of the chromedirver.exe and store it in dirver

driver = webdriver.Chrome("../Python/Clase/chromedriver.exe")

In [5]:
option= Options() # The first thing we are going to do is to define a series of options to work with Selenium

option.add_experimental_option('excludeSwitches', ['enable-automation']) # Hide myself as robot

option.add_experimental_option('useAutomationExtension', False) # Disables the use of the browser automation extension

option.add_argument('--start-maximized') # Start maximized

option.add_argument('user.data-dir=selenium') # Save cookies

option.add_argument('--incognito') # Incognito window

In [10]:
def sleep_(n1, n2, n3):
    """Random number generator, suitable for sleeps to scrape
    Args:
        n1 (int, optional): the minimum number it can return.
        n2 (int, optional): the maximum number it can return.
        n3 (int, optional): the number of decimal places returned.
    Returns:
        int: random int between the first number and the second with the third number of decimal places.
    """
    return round(np.random.randint(n1,n2) + np.random.rand(1)[0], n3)

In [11]:
# Installs the latest version of the ChromeDriver binary that matches the installed version of Chrome

driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get("https://www.cinesa.es/cines/") # Gets the written url

driver.implicitly_wait(30) # Sets an implicit wait time of 30 seconds in the driver object

driver.find_element("css selector", "#onetrust-reject-all-handler").click() # Rejects the cookies

sleep(sleep_(3, 6, 2)) # The program will wait for a random amount of seconds before moving on to the next line of code

# The text attribute is used to retrieve its text content and store it in the cinema_list

cinema_list = driver.find_element("css selector", "#root > div.page-wrapper > div > div.cinema-list").text

In [7]:
split_list = cinema_list.split("\n")

In [8]:
def pick_in_two(lst,num):
    processed_list = [] # Create an empty list to store the processed elements
    for i in range(0, len(lst), num): # Loop through the list with a step of 2
        s = lst[i].lower() # Convert the current element to lowercase
        s = re.sub( # Replace diacritic characters in the string using a regular expression
            r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
            normalize( "NFD", s), 0, re.I
        )
        s = normalize( 'NFC', s) # Normalize the string to the NFC form
        s = s.replace(' ', '-') # Replace spaces with -
        processed_list.append(s) # Append the processed element to the list
    return processed_list # Return the processed list

cinema_url = pick_in_two(split_list,2)

In [9]:
urls = []
for name in cinema_url:
        urls.append(f"https://www.cinesa.es/cines/{name}/")
urls = urls[:-2] # Since the last two are no longer operative

In [16]:
cinesa_urls = pd.DataFrame(urls)
cinesa_urls.to_csv("data/Cinesa_urls.csv")

In [9]:
movie_list = []
driver = webdriver.Chrome(ChromeDriverManager().install())

for url in tqdm(urls):
    try:
        driver.get(url)
        driver.implicitly_wait(30) # Sets an implicit wait time of 30 seconds in the driver object
        driver.find_element("css selector","#onetrust-reject-all-handler").click() # Rejects the cookies
        adress = driver.find_element("css selector", "#root > div.page-wrapper > div > div.cinema-carousel > div").text # Gets the adress
        movie_list.append(adress) # Append the result to a list
        movies = driver.find_element("css selector", "#root > div.page-wrapper > div > div.cinema-details-showtime-picker > div.v-date-showtime-picker > div.v-showtime-picker > ul").text # Gets the movies info
        movie_list.append(movies) # Append the result to a list
        info = driver.find_element("css selector", "#cinemainfo > div.tab-content > div").text # Gets the cinema info
        movie_list.append(info) # Append the result to a list
        driver.find_element("css selector","#cinemainfo > div.v-tabs > header > div > ul > li:nth-child(5) > button").click() # Click the price button
        price = driver.find_element("css selector", "#cinemainfo > div.tab-content").text # Gets the cinema price
        movie_list.append(price) # Append the result to a list
    except:
        adress = driver.find_element("css selector", "#root > div.page-wrapper > div > div.cinema-carousel > div").text # Gets the adress
        movie_list.append(adress) # Append the result to a list
        movies = driver.find_element("css selector", "#root > div.page-wrapper > div > div.cinema-details-showtime-picker > div.v-date-showtime-picker > div.v-showtime-picker > ul").text # Gets the movies info
        movie_list.append(movies) # Append the result to a list
        info = driver.find_element("css selector", "#cinemainfo > div.tab-content > div").text # Gets the cinema info
        movie_list.append(info) # Append the result to a list
        driver.find_element("css selector","#cinemainfo > div.v-tabs > header > div > ul > li:nth-child(5) > button").click() # Click the price button
        price = driver.find_element("css selector", "#cinemainfo > div.tab-content").text # Gets the cinema price
        movie_list.append(price) # Append the result to a list

100%|██████████| 37/37 [18:40<00:00, 30.27s/it]


In [10]:
# Convert the whole list to dataframe

df_movie_list = pd.DataFrame(movie_list)
df_movie_list.head()

,0
0,As Cancelas\nISENSE\nCentro comercial As Cance...
1,Llaman a la puerta\nDurante unas vacaciones en...
2,"CINESA AS CANCELAS, SANTIAGO DE COMPOSTELA (A ..."
3,PRECIOS GENERALES\nLaborables (de lunes a juev...
4,Bahía de Santander\nCentro comercial Bahía de ...


In [11]:
# Convert to csv and store it in data folder with the date of extraction

now = datetime.now()

df_movie_list.to_csv(f"data/Cinesa_cartelera_{now.date()}.csv")

In [13]:
# Installs the latest version of the ChromeDriver binary that matches the installed version of Chrome

driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get("https://yelmocines.es/") # Gets the written url

driver.implicitly_wait(30) # Sets an implicit wait time of 30 seconds in the driver object

driver.find_element("css selector", "#CybotCookiebotDialogBodyButtonDecline").click() # Rejects the cookies

driver.find_element("css selector", "#geo__check").click() # Rejects the cookies

driver.find_element("css selector", "#renderBody_geolocation_geo__popup > div.geo__content > nav > button.btn.btn--decline").click() # Rejects the cookies

driver.implicitly_wait(30) # Sets an implicit wait time of 30 seconds in the driver object

driver.find_element("css selector", "#takeover-close").click() # Rejects the cookies

driver.implicitly_wait(30) # Sets an implicit wait time of 30 seconds in the driver object
cinema_list2 = driver.find_element("css selector", "#ddlCity").text

In [14]:
split_list2 = cinema_list2.split("\n")
cinema_url2 = pick_in_two(split_list2,1)
urls2 = []
for name in cinema_url2:
        if name == "a-coruña":
                urls2.append("https://yelmocines.es/cartelera/a-coruna")
        else:
                urls2.append(f"https://yelmocines.es/cartelera/{name}")
urls2 = urls2[1:] # Since the first one is useless

In [15]:
movie_list2 = []
driver = webdriver.Chrome(ChromeDriverManager().install())

for url in tqdm(urls2):
    try:
        driver.get(url)
        driver.implicitly_wait(30) # Sets an implicit wait time of 30 seconds in the driver object
        driver.find_element("css selector", "#CybotCookiebotDialogBodyButtonDecline").click() # Rejects the cookies
        sleep(3) # The program will wait for a 3 secs
        movies2 = driver.find_element("css selector", "#now__city").text # Gets the movies info
        movie_list2.append(movies2) # Append the result to a list
    except:
        sleep(3) # The program will wait for a 3 secs
        movies2 = driver.find_element("css selector", "#now__city").text # Gets the movies info
        movie_list2.append(movies2) # Append the result to a list

100%|██████████| 23/23 [11:35<00:00, 30.23s/it]


In [16]:
info_list2 = []
driver = webdriver.Chrome(ChromeDriverManager().install())
elements = [
    '#now__city > div:nth-child(3) > h2',
    '#now__city > div:nth-child(4) > h2',
    '#now__city > div:nth-child(5) > h2',
    '#now__city > div:nth-child(6) > h2',
    '#now__city > div:nth-child(7) > h2',
    '#now__city > div:nth-child(8) > h2',
    '#now__city > div:nth-child(9) > h2',
    '#now__city > div:nth-child(10) > h2',
    '#now__city > div:nth-child(11) > h2'
]
try:
    driver.get(urls2[13])
    sleep(2)
    driver.implicitly_wait(30) # Sets an implicit wait time of 30 seconds
    driver.find_element("css selector", "#CybotCookiebotDialogBodyButtonDecline").click() # Rejects the cookies
    sleep(2)
    for element in elements:
        sleep(2)
        driver.find_element("css selector", element).click()
        sleep(2)
        driver.find_element("css selector", element).click()
        sleep(2)
        prueba2 = driver.find_element("css selector", "#wrapper > div.contentWrapper.cf > section.infoComplejo > article > div > div").text
        sleep(2)
        info_list2.append(prueba2) # Append the result to a list
except:
    sleep(2)
    for element in elements:
        sleep(2)
        driver.find_element("css selector", element).click()
        sleep(2)
        driver.find_element("css selector", element).click()
        sleep(2)
        prueba2 = driver.find_element("css selector", "#wrapper > div.contentWrapper.cf > section.infoComplejo > article > div > div").text
        sleep(2)
        info_list2.append(prueba2) # Append the result to a list

In [18]:
info_list3 = []
driver = webdriver.Chrome(ChromeDriverManager().install())
elements = [
    '#now__city > div.divComplejo.now__cinema > h2 > a'

]
numbers = [1]
for i in tqdm(numbers):
    try:
        driver.get(urls2[i])
        sleep(2)
        driver.implicitly_wait(30) # Sets an implicit wait time of 30 seconds
        driver.find_element("css selector", "#CybotCookiebotDialogBodyButtonDecline").click() # Rejects the cookies
        sleep(2)
        for element in elements:
            sleep(2)
            driver.find_element("css selector", element).click()
            sleep(2)
            driver.find_element("css selector", element).click()
            sleep(2)
            prueba2 = driver.find_element("css selector", "#wrapper > div.contentWrapper.cf > section.infoComplejo > article > div > div").text
            sleep(2)
            info_list3.append(prueba2) # Append the result to a list
    except:
        sleep(2)
        for element in elements:
            sleep(2)
            driver.find_element("css selector", element).click()
            sleep(2)
            driver.find_element("css selector", element).click()
            sleep(2)
            prueba2 = driver.find_element("css selector", "#wrapper > div.contentWrapper.cf > section.infoComplejo > article > div > div").text
            sleep(2)
            info_list3.append(prueba2) # Append the result to a list

100%|██████████| 1/1 [00:13<00:00, 13.31s/it]


In [12]:
# Convert the whole list to dataframe

df_movie_list2 = pd.DataFrame(movie_list2)
z = df_movie_list2.iloc[0][0].split("\n")

NameError: name 'movie_list2' is not defined

In [22]:
# Convert to csv and store it in data folder with the date of extraction

now = datetime.now()

df_movie_list2.to_csv(f"data/Yelmo_cartelera_{now.date()}.csv")

In [17]:
# Installs the latest version of the ChromeDriver binary that matches the installed version of Chrome

driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get("https://www.golem.es/") # Gets the written url

driver.implicitly_wait(30) # Sets an implicit wait time of 30 seconds in the driver object

driver.find_element("css selector", "#cookiescript_reject").click() # Rejects the cookies

driver.find_element("css selector", "#css3menu1 > li > a").click() # Open table

cinema_list3 = driver.find_element("css selector", "#menuCines > table > tbody > tr:nth-child(3)").text

In [18]:
split_list3 = cinema_list3.split("\n")
cinema_url3 = pick_in_two(split_list3,1)
urls3 = []
for name in cinema_url3:
    if name == "golem":
        urls3.append(f"https://www.golem.es/golem/golem-madrid")
    elif name == "van-golem":
        urls3.append(f"https://www.golem.es/golem/vangolem")
    else:
        urls3.append(f"https://www.golem.es/golem/{name}")

In [20]:
golem_urls = pd.DataFrame(urls3)
golem_urls.to_csv("data/Golem_urls.csv")

In [96]:
movie_list3 = []
driver = webdriver.Chrome(ChromeDriverManager().install())

for url in tqdm(urls3):
    try:
        driver.get(url)
        driver.implicitly_wait(30) # Sets an implicit wait time of 30 seconds in the driver object
        adress3 = driver.find_element("css selector", "#derechaInfoCine > table > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(2) > td.txtNegMedio").text # Gets the cinema info
        movie_list3.append(adress3) # Append the result to a list
        movies3 = driver.find_element("css selector", "body > table > tbody > tr:nth-child(2) > td > table > tbody > tr:nth-child(2) > td > table > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(1) > table > tbody > tr:nth-child(1) > td > table > tbody > tr:nth-child(3) > td > table").text # Gets the movies info
        movie_list3.append(movies3) # Append the result to a list
        price3 = driver.find_element("css selector", "#derechaInfoCine > table > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(1) > td").text # Gets the cinema price
        movie_list3.append(price3) # Append the result to a list
    except:
        adress3 = driver.find_element("css selector", "#derechaInfoCine > table > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(2) > td.txtNegMedio").text # Gets the cinema info
        movie_list3.append(adress3) # Append the result to a list
        movies3 = driver.find_element("css selector", "body > table > tbody > tr:nth-child(2) > td > table > tbody > tr:nth-child(2) > td > table > tbody > tr:nth-child(3) > td > table > tbody > tr > td:nth-child(1) > table > tbody > tr:nth-child(1) > td > table > tbody > tr:nth-child(3) > td > table").text # Gets the movies info
        movie_list3.append(movies3) # Append the result to a list
        price3 = driver.find_element("css selector", "#derechaInfoCine > table > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(1) > td").text # Gets the cinema price
        movie_list3.append(price3) # Append the result to a list

100%|██████████| 6/6 [00:11<00:00,  1.97s/it]


In [100]:
# Convert the whole list to dataframe

df_movie_list3 = pd.DataFrame(movie_list3)

In [98]:
# Convert to csv and store it in data folder with the date of extraction

now = datetime.now()

df_movie_list3.to_csv(f"data/Golem_cartelera_{now.date()}.csv")

In [21]:
# Installs the latest version of the ChromeDriver binary that matches the installed version of Chrome

driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get("https://www.cinesrenoir.com/") # Gets the written url

driver.implicitly_wait(30) # Sets an implicit wait time of 30 seconds in the driver object

driver.find_element("css selector", "#band-cookies-ok").click() # Accepts the cookies

sleep(sleep_(2, 3, 2)) # The program will wait for a random amount of seconds before moving on to the next line of code

driver.find_element("css selector", "#bigshopNav > div.classy-menu > div.classynav > ul > li:nth-child(1)").click() # Display Menu

sleep(sleep_(2, 3, 2))

# The text attribute is used to retrieve its text content and store it in the cinema_list

cinema_list4 = driver.find_element("css selector", "#bigshopNav > div.classy-menu > div.classynav > ul > li:nth-child(1) > ul").text

In [22]:
split_list4 = cinema_list4.split("\n")
sp_list = [x.split(" (")[0] for x in split_list4]
cinema_url4 = pick_in_two(sp_list,1)
urls4 = []
for name in cinema_url4:
    if name == "renoir-plaza-de-españa":
        urls4.append("https://www.cinesrenoir.com/cine/renoir-plaza-de-espana/cartelera/")
    else:
        urls4.append(f"https://www.cinesrenoir.com/cine/{name}/cartelera/")

In [24]:
renoir_urls = pd.DataFrame(urls4)
renoir_urls.to_csv("data/Renoir_urls.csv")

In [40]:
movie_list4 = []
driver = webdriver.Chrome(ChromeDriverManager().install())

for url in tqdm(urls4):
    try:
        driver.get(url)
        driver.implicitly_wait(30) # Sets an implicit wait time of 30 seconds in the driver object
        driver.find_element("css selector", "#band-cookies-ok").click() # Accepts the cookies
        adress4 = driver.find_element("css selector", "body > section.single_product_details_area.section_padding_50 > div > div > div.d-none.d-md-block.col-12.col-md-7 > div").text # Gets the cinema info
        movie_list4.append(adress4) # Append the result to a list
        movies4 = driver.find_element("css selector", "body > section.shop_list_area.mt-2 > div > div:nth-child(2)").text # Gets the movies info
        movie_list4.append(movies4) # Append the result to a list
        sleep(2)
        driver.find_element("css selector", "body > section.single_product_details_area.section_padding_50 > div > div > div.col-12.col-md-5.mb-15 > a:nth-child(3)").click()
        sleep(2)
        price4 = driver.find_element("css selector", "body > section:nth-child(3)").text # Gets the cinema price
        movie_list4.append(price4) # Append the result to a list
    except:
        adress4 = driver.find_element("css selector", "body > section.single_product_details_area.section_padding_50 > div > div > div.d-none.d-md-block.col-12.col-md-7 > div").text # Gets the cinema info
        movie_list4.append(adress4) # Append the result to a list
        movies4 = driver.find_element("css selector", "body > section.shop_list_area.mt-2 > div > div:nth-child(2)").text # Gets the movies info
        movie_list4.append(movies4) # Append the result to a list
        sleep(2)
        driver.find_element("css selector", "body > section.single_product_details_area.section_padding_50 > div > div > div.col-12.col-md-5.mb-15 > a:nth-child(3)").click()
        sleep(2)
        price4 = driver.find_element("css selector", "body > section:nth-child(3)").text # Gets the cinema price
        movie_list4.append(price4) # Append the result to a list

100%|██████████| 4/4 [01:53<00:00, 28.28s/it]


In [41]:
# Convert the whole list to dataframe

df_movie_list4 = pd.DataFrame(movie_list4)
z = df_movie_list4.iloc[0][0].split("\n")

,0
0,Cines Princesa (11 salas)\nDirección\nPRINCESA...
1,AFTERSUN\nde Charlotte Wells\nVersión Original...
2,Precios para no socios\nNOMBRE VALIDEZ TAQUILL...
3,Renoir Floridablanca (7 salas)\nDirección\nFLO...
4,AFTERSUN\nde Charlotte Wells\nVersión Original...
5,Precios para no socios\n\nNOMBRE VALIDEZ TAQUI...
6,Renoir Plaza de España (5 salas)\nDirección\nM...
7,AS BESTAS\nde Rodrigo Sorogoyen\nVersión Origi...
8,Precios para no socios\nNOMBRE VALIDEZ TAQUILL...
9,"Renoir Retiro (4 salas)\nDirección\nNARVÁEZ, 4..."


In [42]:
# Convert to csv and store it in data folder with the date of extraction

now = datetime.now()

df_movie_list4.to_csv(f"data/Renoir_cartelera_{now.date()}.csv")

In [33]:
df_movie_list = pd.read_csv("data/Cinesa_cartelera_2023-02-11.csv")

In [34]:
df_movie_list.drop(["Unnamed: 0"], axis = 1, inplace=True)

In [ ]:
def get_adress(lst):
    coord = []
    geoloca=Nominatim(user_agent="juan")
    for element in range(0,len(lst)):
        try:
            ad = lst[element][-2]+", "+lst[element][-1]
            location= geoloca.geocode(ad)
            coord.append(location[-1])
        except:
            for part in lst[element]:
                if "Centro" in part:
                    try:
                        location= geoloca.geocode(part)
                        coord.append(location[-1])
                    except:
                        print(part)
                        location= geoloca.geocode(part[1])
                        coord.append(location[-1])
                else:
                    pass
    return coord

In [ ]:
geoloca=Nominatim(user_agent="juan")
location= geoloca.geocode("Heron City Las Rozas")
location

Location(Heron City Madrid, La Dehesa de Navalcarbón, Las Rozas de Madrid, Área metropolitana de Madrid y Corredor del Henares, Comunidad de Madrid, España, (40.5160134, -3.897459259461641, 0.0))

In [ ]:
adress = df_movie_list[0][::4].apply(lambda x: x.split("\n"))
adress1 = adress[0][-2]+", "+adress[0][-1]
adress1

'Avenida Do Camiño Francés, 3, 15703 Santiago de Compostela, A Coruña'